# Create Tools

tools 可以是常规的 python 函数

In [1]:
from autogen import ConversableAgent
import autogen
import json

# 从 JSON 文件中加载配置
with open('config.json', 'r') as f:
    config = json.load(f)

API_KEY = config["API_KEY"]
BASE_URL = config["BASE_URL"]

llm_config = {"model": "glm-4", "api_key": API_KEY, "base_url":BASE_URL}

In [2]:
# 示例 （计算器）
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

## Registering tools

定义好的 `Agent` 类型可以通过 `register_for_llm` 方法注册工具

In [3]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

tools 需要至少向两个 agent 注册才能在对话中发挥作用

通过 `register_for_llm` 注册的 agent 可以调用该工具，通过 `register_for_execution` 注册才可以执行该工具

or

通过 `autogen.register_function` 同时向两个 agent 注册工具

In [4]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

## Use tools

zhipu 是真的蠢

In [6]:
chat_result = user_proxy.initiate_chat(assistant, message="calculate the (44232 + 13312 / (232 - 32)) * 5 ?")

User (to Assistant):

calculate the (44232 + 13312 / (232 - 32)) * 5 ?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 08-28 10:17:35] {329} WARNING - Model glm-4 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Assistant (to User):


***** Suggested tool call (call_20240828101735d9238712086d48d8): calculator *****
Arguments: 
{"a":44232,"b":13312,"operator":"/"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_20240828101735d9238712086d48d8) *****
3
****************************************************************************

------------------------------------

In [11]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]